In [1]:

import datetime as dt
import requests
import pandas as pd
import numpy as np
import matplotlib
import mplfinance as mpf
import talib

In [2]:

def now(lookback): 
    #get time now convert it to unix
    timenow = dt.datetime.utcnow()
    end_time = timenow.replace(tzinfo=dt.timezone.utc).timestamp()
    #get lookback and convert to unix 
    start_time = dt.datetime.utcnow() - dt.timedelta(lookback)
    start_time = start_time.replace(tzinfo=dt.timezone.utc).timestamp() 
    return start_time, end_time

_start_time, _end_time = now(3)

data = requests.get(f"https://ftx.com/api//markets/{'BTC-PERP'}/candles?resolution={3600}&start_time={_start_time}&end_time={_end_time}").json()

data = pd.DataFrame(data['result'])

data['time'] = pd.to_datetime(data['time'], unit='ms')
data.set_index('time', inplace= True)
#print(data.head(5))

In [3]:
def getmarkets(PERP):
    markets = requests.get(f"https://ftx.com/api/markets").json()
    markets = pd.DataFrame(markets['result'])
    markets = markets['name'].tolist()
    markets = [market for market in markets if market.endswith(PERP)]
    return markets

perpnames = getmarkets('PERP')
print(perpnames)

['1INCH-PERP', 'AAVE-PERP', 'ADA-PERP', 'AGLD-PERP', 'ALCX-PERP', 'ALGO-PERP', 'ALICE-PERP', 'ALPHA-PERP', 'ALT-PERP', 'AMPL-PERP', 'ANC-PERP', 'APE-PERP', 'AR-PERP', 'ASD-PERP', 'ATLAS-PERP', 'ATOM-PERP', 'AUDIO-PERP', 'AVAX-PERP', 'AXS-PERP', 'BADGER-PERP', 'BAL-PERP', 'BAND-PERP', 'BAO-PERP', 'BAT-PERP', 'BCH-PERP', 'BIT-PERP', 'BNB-PERP', 'BNT-PERP', 'BOBA-PERP', 'BRZ-PERP', 'BSV-PERP', 'BTC-PERP', 'BTT-PERP', 'C98-PERP', 'CAKE-PERP', 'CEL-PERP', 'CELO-PERP', 'CHR-PERP', 'CHZ-PERP', 'CLV-PERP', 'COMP-PERP', 'CONV-PERP', 'CREAM-PERP', 'CRO-PERP', 'CRV-PERP', 'CUSDT-PERP', 'CVC-PERP', 'CVX-PERP', 'DASH-PERP', 'DAWN-PERP', 'DEFI-PERP', 'DENT-PERP', 'DODO-PERP', 'DOGE-PERP', 'DOT-PERP', 'DRGN-PERP', 'DYDX-PERP', 'EDEN-PERP', 'EGLD-PERP', 'ENJ-PERP', 'ENS-PERP', 'EOS-PERP', 'ETC-PERP', 'ETH-PERP', 'EXCH-PERP', 'FIDA-PERP', 'FIL-PERP', 'FLM-PERP', 'FLOW-PERP', 'FTM-PERP', 'FTT-PERP', 'GALA-PERP', 'GMT-PERP', 'GRT-PERP', 'HBAR-PERP', 'HNT-PERP', 'HOLY-PERP', 'HOT-PERP', 'HT-PERP', 'HUM-PE

In [4]:
def ohlc(perpnames, tf, daysback):
    _start_time, _end_time = now(daysback) 
    data = requests.get(f"https://ftx.com/api//markets/{perpnames}/candles?resolution={tf}&start_time={_start_time}&end_time={_end_time}").json()
    data = pd.DataFrame(data['result'])
    return data

In [5]:
dfs = []
tf = 14400
daysback = 7
for perp in perpnames: 
    dfs.append(ohlc(perp, tf, daysback))

In [20]:
all = pd.concat(dict(zip(perpnames,dfs)), axis=1)
#closes = all.loc[:,all.columns.get_level_values(1).isin(['close'])]

nameless = pd.concat(dfs, axis=1)
closes = nameless.loc[:,nameless.columns.get_level_values(0).isin(['close'])]
closes

,close,close,close,close,close,close,close,close,close,close,...,close,close,close,close,close,close,close,close,close,close
0,1.3176,116.60,0.788550,0.9565,94.0,0.7015,5.8125,0.26405,3112.65,0.9318,...,0.09355,0.183600,170.59,0.803175,2.9827,18520.0,1811.00,153.50,0.04054,0.48825
1,1.3474,118.93,0.791815,0.9580,94.8,0.7079,5.8575,0.26765,3141.35,0.9433,...,0.09480,0.185625,174.33,0.815300,3.0198,18620.0,1843.75,154.75,0.04170,0.49150
2,1.3399,121.47,0.797315,0.9520,94.4,0.7104,5.8575,0.26860,3143.60,0.9433,...,0.09480,0.185525,174.94,0.796325,3.0335,18615.0,1839.50,151.76,0.04103,0.49275
3,1.3358,119.80,0.792380,0.9515,94.0,0.7115,5.8775,0.27020,3133.40,0.9443,...,0.09440,0.183650,179.68,0.785700,3.0485,18660.0,1847.75,145.26,0.04092,0.49425
4,1.3521,118.90,0.793180,0.9690,94.5,0.7121,5.8750,0.26840,3140.85,0.9527,...,0.09460,0.184600,183.26,0.790300,3.0484,18690.0,1859.25,148.11,0.04097,0.49750
5,1.3461,120.06,0.793905,1.0400,93.6,0.7056,5.8750,0.26945,3140.10,0.9503,...,0.09440,0.182775,185.68,0.790450,3.0508,18445.0,1852.00,152.56,0.04089,0.49725
6,1.3317,120.34,0.789175,0.9730,92.4,0.7048,5.8250,0.26585,3115.65,0.9433,...,0.09340,0.184975,185.36,0.785350,3.0246,18370.0,1826.25,151.62,0.04055,0.49500
7,1.3786,120.42,0.798255,1.0085,93.2,0.7153,5.9675,0.27175,3150.75,0.9478,...,0.09455,0.188550,193.03,0.797275,3.0759,18620.0,1871.25,152.33,0.04124,0.50250
8,1.3780,119.11,0.793665,0.9715,92.6,0.7105,5.8675,0.26850,3137.25,0.9478,...,0.09385,0.186075,189.35,0.797975,3.0632,18460.0,1846.50,144.96,0.04092,0.50025
9,1.3698,117.67,0.788955,0.9550,91.9,0.7026,5.8275,0.26775,3111.50,0.9466,...,0.09305,0.187050,185.06,0.791500,3.0215,18390.0,1826.00,147.06,0.04014,0.49625


In [21]:
def divs(closes, columns, ob=70, os=30, period=14):
    """Calculates bullish and bearish RSI divergences under oversold or overbought conditions"""

    closes['RSI'] = talib.RSI(closes.iloc[:, columns])
    closes['rolling_rsi_high'] = closes['RSI'].rolling(period).max()
    closes['rolling_rsi_low'] = closes['RSI'].rolling(period).min()
    closes['rolling_closing_high'] = closes.iloc[:, columns].rolling(period).max()
    closes['rolling_closing_low'] = closes.iloc[:, columns].rolling(period).min()


    closes['new_RSI_high'] = np.where(closes['rolling_rsi_high'] > closes['rolling_rsi_high'].shift(), 1, 0)
    closes['new_RSI_low'] = np.where(closes['rolling_rsi_low'] < closes['rolling_rsi_low'].shift(), 1, 0)


    closes['new_closing_high'] = np.where(closes['rolling_closing_high'] > closes['rolling_closing_high'].shift(), 1, 0)
    closes['new_closing_low'] = np.where(closes['rolling_closing_low'] < closes['rolling_closing_low'].shift(), 1, 0)

    bearishrsidiv = np.where((closes['new_closing_high'] == 1) & (closes['new_RSI_high'] == 0) & (closes['RSI'] > ob), 1, 0)
    bullishrsidiv = np.where((closes['new_closing_low'] == 1) & (closes['new_RSI_low'] == 0) & (closes['RSI'] < os), 1, 0)
   # closes.insert(1, 'bearish_rsi_div', bearishrsidiv)
   # closes.insert(2, 'bullish_rsi_div', bullishrsidiv)
    closes['bearish_rsi_div'] = np.where((closes['new_closing_high'] == 1) & (closes['new_RSI_high'] == 0) & (closes['RSI'] > ob), 1, 0)
    closes['bullish_rsi_div'] = np.where((closes['new_closing_low'] == 1) & (closes['new_RSI_low'] == 0) & (closes['RSI'] < os), 1, 0)
    closes = closes.dropna()
    return closes[['RSI','bullish_rsi_div', 'bearish_rsi_div']]


In [22]:
closes.iloc[-1:,]


,close,close,close,close,close,close,close,close,close,close,...,close,close,close,close,close,close,close,close,close,close
41,1.6066,161.98,0.8538,1.506,98.2,0.7564,6.255,0.29145,3360.7,1.0519,...,0.10005,0.1926,193.3,0.7949,3.1423,20465.0,2150.0,164.96,0.0424,0.53825


In [23]:
rsidata = [] 
for column in range(179):
   rsidata.append(divs(closes, column))
alle = pd.concat(dict(zip(perpnames,rsidata)), axis=1)
øve = pd.concat(rsidata, axis = 1)
øve


C:\Users\jonas\AppData\Local\Temp\ipykernel_3752\1604535116.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closes['RSI'] = talib.RSI(closes.iloc[:, columns])
C:\Users\jonas\AppData\Local\Temp\ipykernel_3752\1604535116.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closes['rolling_rsi_high'] = closes['RSI'].rolling(period).max()
C:\Users\jonas\AppData\Local\Temp\ipykernel_3752\1604535116.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

,RSI,bullish_rsi_div,bearish_rsi_div,RSI,bullish_rsi_div,bearish_rsi_div,RSI,bullish_rsi_div,bearish_rsi_div,RSI,...,bearish_rsi_div,RSI,bullish_rsi_div,bearish_rsi_div,RSI,bullish_rsi_div,bearish_rsi_div,RSI,bullish_rsi_div,bearish_rsi_div


In [24]:
cols = (alle.iloc[-3:] != 0).any()
websitedata = alle.iloc[-3:][cols[cols].index]
websitedata

Empty DataFrame
Columns: []
Index: []

In [25]:
div_df = pd.DataFrame()
def get_divs(øve):
   øve = øve.iloc[-1:]
   div_df['dives'] = np.where(øve.iloc[-1:] == 0, True, False)
   return div_df